### TODO:: 

1. build date function and convert utc to mm dd - done!!!

    1.1 - instead of using date select miliseconds as test data from 11.2018 / check if at least 15% of data - done!!!


2. remove duplicate items per single person / remove by bidid  - done my michael - done!!!


3. remove trash columns from android_bids

    3.1 remove marketplace col - done !!!
    
    3.2 device_maker and device_model NaN's changed to unknown and claasified as new device_maker or device_model - done!!!
    
    3.3 
    
    

4. remove trash columns from play_apps

    4.1 Create dataframe from files - done!!!
    
    4.2 Do we need to use play_apps -> Which cols do we want to use??
    
        [video, category, score, histogram, reviews, editors_choice, (price, free), iap, size, installs, content_rating, iap_range, interactive_elements?, bids]


5. trun into comma seperated label for play_apps -> category


6. Merge data from two tables android_bids_us and play_apps

    6.1 USE pandas merge datafreames function to merge both files

In [2]:
#Install once
#!pip install imbalanced-learn --user
#!pip install delayed --user

#imports
import pandas as pd
import numpy as np
from datetime import datetime
from zipfile import ZipFile
import pickle
from pathlib import Path


from sklearn.preprocessing import LabelEncoder, StandardScaler, MultiLabelBinarizer


In [3]:
#Read orig android data
origdata = pd.read_csv('data/android_bids_us.csv')
origdata.head()


,bidid,utc_time,app_id,user_state,user_isp,device_maker,device_model,device_osv,device_height,device_width,marketplace,click
0,87b0108c-6e9f-4783-8b80-8dd5aa3529a8,1536444331894,com.OppanaGames.CarSim,TX,AT&T Wireless,NaN,NaN,7.0,720,1280,chartboost,0
1,9284441f-a6ea-4698-9017-86436b92d416,1536444363932,com.mobilityware.CrownSolitaire,TX,T-Mobile USA,NaN,NaN,7.0,1280,720,chartboost,0
2,7e4779c2-f757-4324-8c2a-138b558b5a54,1536444386908,com.kuttigames.tenkyurollball,LA,AT&T Wireless,NaN,NaN,7.1.1,720,1280,chartboost,0
3,b64ea208-38ca-42ac-895e-0842d7352cc3,1536444405023,com.trendactionfree.call_of_sniper_duty_frontl...,TN,Comcast Cable,NaN,NaN,6.0.1,720,1280,chartboost,0
4,fd0c07cb-31f6-408c-9315-1cb652e76abc,1536444443458,com.landslab.my3rdgrademathgames,NC,AT&T U-verse,NaN,NaN,7.1.1,1024,768,chartboost,0


# Analyze android dataset

In [4]:
dataTypes = origdata.dtypes
print(f'Total columns: {dataTypes.size} ||| Total rows: {origdata.count()}')
print(f'Data type of each column of Dataframe : ')
print(dataTypes)
# We may have NAN's in user_isp, device_maker, device_model

print('-----------------------------------')
print('Amount of NaN under utc_time', origdata['utc_time'].isnull().sum())
# all items in marketplace col equal 'chartboost'. we can remove col

#Print start time 
start_time = origdata['utc_time'].min()
end_time = origdata['utc_time'].max()
print('Start Time: ',  datetime.fromtimestamp(start_time/1000))
print('End Time: ',  datetime.fromtimestamp(end_time/1000))

print('-----------------------------------')

print('-----------------------------------')
print('Variance data of android dataset')
print(origdata.var(ddof=0))
print('-----------------------------------')


print('-----------------------------------')
print(origdata.marketplace.unique())
# all items in marketplace col equal 'chartboost'. we can remove col
print('-----------------------------------')

print('-----------------------------------')
print('app_id::')
print('Amount of NaN under app_id', origdata['app_id'].isnull().sum())
print(origdata.app_id.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('user_isp::')
print('Amount of NaN under user_isp', origdata['user_isp'].isnull().sum())
print(origdata.user_isp.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('user_state::')
print(origdata.user_state.value_counts())
#Consider use onehotencoding insted of laelencoding
print('-----------------------------------')

print('-----------------------------------')
print('device_maker::')
print(origdata.device_maker.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('device_model::')
print(origdata.device_model.value_counts())
print('-----------------------------------')



Total columns: 12 ||| Total rows: bidid            3148828
utc_time         3148828
app_id           3148828
user_state       3148828
user_isp         3148789
device_maker     2184824
device_model     2184970
device_osv       3148828
device_height    3148828
device_width     3148828
marketplace      3148828
click            3148828
dtype: int64
Data type of each column of Dataframe : 
bidid            object
utc_time          int64
app_id           object
user_state       object
user_isp         object
device_maker     object
device_model     object
device_osv       object
device_height     int64
device_width      int64
marketplace      object
click             int64
dtype: object
-----------------------------------
Amount of NaN under utc_time 0
Start Time:  2018-08-06 21:00:20.974000
End Time:  2018-11-19 14:24:53.941000
-----------------------------------
-----------------------------------
Variance data of android dataset
utc_time         5.383583e+18
device_height    2.694544e+05


# Prepare android dataset

In [10]:
#Rearrange dataframe
#clone orig dataframe
android_df = origdata.copy()

#-----------------------------

#Add date from utc and remove utc
#android_df['date'] = android_df['utc_time'].apply(lambda x: datetime.fromtimestamp(x/1000))
#android_df = android_df.drop(['utc_time'], axis=1)

# this col will be removed in the last part after splitting the data

#-----------------------------

#Remove marketplace col -> all values are the same
android_df = android_df.drop(['marketplace'], axis=1)

#-----------------------------

#Remove duplicated bidid. Keep only single instance keep the one with click == 1
android_df = android_df[~((android_df['bidid'].duplicated(keep=False))&(android_df['click']==0))]
#and then remove bids column
android_df = android_df.drop(['bidid'], axis=1)

#-----------------------------

#-----------------------------
#Change device_maker NaN's to unknown manufacturerer
#android_df['device_maker'] = android_df['device_maker'].apply(lambda x: 'unknown' if x!=x else x)
android_df.drop('device_maker', axis = 1,inplace = True)

#Change device_model NaN's to unknown valid model
#android_df['device_model'] = android_df['device_model'].apply(lambda x: 'unknown' if x!=x else x)
android_df.drop('device_model', axis = 1,inplace = True)
#-----------------------------

#-----------------------------
# Remove device_osv column
android_df.drop('device_osv', axis = 1,inplace = True)
#-----------------------------

#-----------------------------
# Take size cols and transform into resolution
android_df['screen_resolution'] = android_df['device_height']*android_df['device_width']
android_df.drop(['device_width', 'device_height'], axis=1, inplace=True)

# And then do scaling to new col
scaler = StandardScaler()
android_df['screen_resolution'] = scaler.fit_transform(android_df[['screen_resolution']])
#-----------------------------

#-----------------------------
# LabelEncoder for user_state
#Consider use onehotencoding insted of laele ncoding
le1 = LabelEncoder()
android_df['user_state'] = le1.fit_transform(android_df['user_state'])
#-----------------------------


#-----------------------------
# user_isp
# LabelEncoder for user_isp

#Consider use onehotencoding insted of laele ncoding
#android_df['user_isp'] = android_df['user_isp'].fillna(method="ffill")
#le2 = LabelEncoder()
#android_df['user_isp'] = le2.fit_transform(android_df['user_isp'])

android_df.drop('user_isp', axis = 1,inplace = True)
#-----------------------------


android_df

,utc_time,app_id,user_state,click,screen_resolution
0,1536444331894,com.OppanaGames.CarSim,43,0,-0.530933
1,1536444363932,com.mobilityware.CrownSolitaire,43,0,-0.530933
2,1536444386908,com.kuttigames.tenkyurollball,18,0,-0.530933
3,1536444405023,com.trendactionfree.call_of_sniper_duty_frontl...,42,0,-0.530933
5,1536444487429,com.kick.trucks.manual.shift.driving,9,0,-0.530933
...,...,...,...,...,...
3148822,1541472443393,com.qre.Coast.Lifeguard.Beach.Rescue,43,0,-0.530933
3148823,1541472458853,com.altitude.rrsv,43,0,-0.530933
3148825,1541472516760,br.com.tapps.penguinevolution,24,0,-0.530933
3148826,1541472552347,air.com.FDGEntertainment.Lonewolf.gp,48,0,0.857330


In [11]:
print(f'Click percent in data: {android_df.click.sum()/android_df.shape[0]*100}')


Click percent in data: 7.259269146156809


# Extract play_apps data and first clean

In [12]:
# Get play_apps data


#Read orig play_apps data
#app_details_orig = ZipFile("data/play_apps.zip")
#app_file = 'play_apps/a008.com.fc2.blog.androidkaihatu.datecamera2'
#app = pickle.loads(app_details_orig.read(app_file))
#app


def zipToDataFrame(zipped, to_remove):
    rows = []
    
    for f in zipped.filelist:
        app_file = f.filename
        try:
            app = pickle.loads(zipped.read(app_file))
        except EOFError as e:
            continue
        except pickle.UnpicklingError as e:
            continue
        except Exception as e:
            return
            
        # Remove columns
        for col in to_remove:
            del app[col]
        rows.append(app)
    
    return pd.DataFrame(rows)


app_details = ZipFile("data/play_apps.zip")
cols_to_remove = ['title','screenshots', 'icon', 'size',
                  'histogram', 'description', 'description_html', 
                  'recent_changes', 'developer', 'developer_id', 'developer_email', 'developer_url',
                  'developer_address', 'url', 'current_version', 'updated', 'required_android_version',
                  'iap_range']


orig_play_apps = zipToDataFrame(app_details, cols_to_remove)
orig_play_apps



,video,category,score,reviews,editors_choice,price,free,iap,installs,content_rating,interactive_elements,app_id,bids
0,None,[PHOTOGRAPHY],4.0,435,False,0,True,False,"100,000+",[Everyone],None,a008.com.fc2.blog.androidkaihatu.datecamera2,484
1,None,[TOOLS],4.6,1432,False,0,True,False,"50,000+",[Everyone],None,a201706011153.xsky.txvpn,53
2,None,[PRODUCTIVITY],4.6,1362,False,0,True,False,"50,000+",[Everyone],None,a201706021616.vpn.turbovpn,43
3,https://www.youtube.com/embed/QulTvgUvuJ0,[TOOLS],3.9,18,False,0,True,False,"1,000+",[Everyone],None,a201707.grmo.a8bit.jp.beautytimer,60
4,None,[TOOLS],4.2,1112,False,0,True,False,"100,000+",[Everyone],None,a2x.studio.fast.charging.battery.supercharging,600
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29458,https://www.youtube.com/embed/JGXTEQbKUPo,[GAME_ACTION],4.5,2512572,True,0,True,True,"50,000,000+","[Teen, Violence, Blood]",[Digital Purchases],zombie.survival.craft.z,34140
29459,None,[ENTERTAINMENT],3.9,172,False,0,True,False,"10,000+",[Everyone],None,zombi.maps.forminecraftpe,248
29460,None,[GAME_ARCADE],4.4,2100,False,0,True,False,"500,000+",[Everyone],None,zone.super.jungle.worldrun2018,134
29461,https://www.youtube.com/embed/9_bq-wQkuWI,[GAME_PUZZLE],4.6,26944,False,0,True,True,"1,000,000+",[Everyone],[Digital Purchases],zoo.rescue,1137


# Analyze play_apps dataset

In [13]:
# explore play_apps data characteristics

apps_dataTypes = orig_play_apps.dtypes

print(f'Total columns: {apps_dataTypes.size} ||| Total rows: {orig_play_apps.count()}')
print(f'Data type of each column of Dataframe : ')
print(apps_dataTypes)
# We may have NAN's in iap_range, interactive_elements, price

print('-----------------------------------')
print('Amount of NaN under price', orig_play_apps['price'].isnull().sum())
print('Amount of NaN under free', orig_play_apps['free'].isnull().sum())
print('price values: ', orig_play_apps.price.unique())
print('free values: ',orig_play_apps.free.unique())

print('How many free from all dataset: ', (orig_play_apps['free'] == True).sum())
# items in price and free should be in a single col
# we can take the free col since the non free item is very little : about 1000 out of 29000
# or we can take col price since 0 represnts free
print('-----------------------------------')

print('-----------------------------------')
print('Variance data of play aps dataset')
print(orig_play_apps.var(ddof=0))
print('-----------------------------------')

print('-----------------------------------')
# items in price and free should be in a single col
print('Amount of NaN under score', orig_play_apps['score'].isnull().sum())
print('score: ', orig_play_apps.score.unique())
print('-----------------------------------')


print('-----------------------------------')
# Test reviews col - need to be scaled
print('Amount of NaN under reviews', orig_play_apps['reviews'].isnull().sum())
print('reviews: ', orig_play_apps.reviews.unique())
print('-----------------------------------')

print('-----------------------------------')
# Removed for testing
# Test bids col - need to be scaled
print('Amount of NaN under bids', orig_play_apps['bids'].isnull().sum())
print('bids: ', orig_play_apps.bids.unique())
print('-----------------------------------')

print('-----------------------------------')
# Removed for testing
print('content_rating column::: ')
# Test content_rating col
rating = orig_play_apps['content_rating'].agg(lambda x: ':'.join(map(str, x)))
print(rating.unique())
# We might drop this column since its require a lot of handling and im not sure its gonna impact our model
print('-----------------------------------')


print('-----------------------------------')
print('category column::: ')
# Test category col
category = orig_play_apps['category'].agg(lambda x: ':'.join(map(str, x)))
print(category.unique())
# We might drop this column since its require a lot of handling and im not sure its gonna impact our model
print('-----------------------------------')

print('-----------------------------------')
print('interactive_elements column::: ')
# Test category col
print('Amount of NaN under interactive_elements', orig_play_apps['interactive_elements'].isnull().sum())
interactive = orig_play_apps['interactive_elements'].agg(lambda x: ':'.join(map(str, x)))
interactive
#print(interactive.unique())
# We might drop this column since its require a lot of handling and im not sure its gonna impact our model
print('-----------------------------------')


Total columns: 13 ||| Total rows: video                    9150
category                29463
score                   29351
reviews                 29463
editors_choice          29463
price                   28312
free                    29463
iap                     29463
installs                29462
content_rating          29463
interactive_elements    11271
app_id                  29463
bids                    29463
dtype: int64
Data type of each column of Dataframe : 
video                   object
category                object
score                   object
reviews                  int64
editors_choice            bool
price                   object
free                      bool
iap                       bool
installs                object
content_rating          object
interactive_elements    object
app_id                  object
bids                     int64
dtype: object
-----------------------------------
Amount of NaN under price 1151
Amount of NaN under free 0
price value

# Prepare play_apps dataset

In [16]:
#Fix and manipulate play_apps data before merge with android_bids_us
#clone orig dataframe
play_apps_df = orig_play_apps.copy()

# Convert String to int.
def convert_installs(str_val=''):
    str_val = str_val.translate({ord(i):None for i in ',+'})
    return int(str_val)

# Convert String to float.
def convert_price(str_val=''):
    str_val = str_val.translate({ord(i):None for i in '$'})
    return float(str_val)


#---------------------------------------------------
# Handle 'Installs' column
# Translate to number and then scale it
play_apps_df['installs'].fillna(method='pad', inplace=True)
play_apps_df['installs'] = play_apps_df['installs'].apply(lambda s: convert_installs(s))

# And then do scaling to new col
scaler = StandardScaler()
play_apps_df['installs'] = scaler.fit_transform(play_apps_df[['installs']])
#---------------------------------------------------


#---------------------------------------------------
# Handle 'video' column
play_apps_df['video'] = play_apps_df['video'].apply(lambda x: 0 if pd.isna(x) else 1)
#---------------------------------------------------

#---------------------------------------------------
# Handle 'score ' column
# Translate to number
play_apps_df['score'] = pd.to_numeric(play_apps_df['score'],errors = 'coerce')
play_apps_df['score'].fillna(method='pad', inplace=True)
play_apps_df['score'] = scaler.fit_transform(play_apps_df[['score']])
#---------------------------------------------------

#---------------------------------------------------
# Handle 'reviews' column - we cant know weither the review is good or bad, 
# but the amount of reviews can point on popularity off the app
# There are no NaN's
# Translate to number
scaler = StandardScaler()
play_apps_df['reviews'] = scaler.fit_transform(play_apps_df[['reviews']])
#---------------------------------------------------


#---------------------------------------------------
#price and free can be merged into single col named price since 0 represntes free app
# and scale it
play_apps_df['price'] = play_apps_df['price'].fillna(method='pad', inplace=False)
play_apps_df['price'] = play_apps_df['price'].apply(lambda s: convert_price(s))
play_apps_df['price'] = scaler.fit_transform(play_apps_df[['price']])
play_apps_df.drop(['free'], axis=1, inplace=True)
#---------------------------------------------------


#---------------------------------------------------
# 'category' column transform to multil-lable
# Removed for testing
#mlb = MultiLabelBinarizer()
#mlb.fit(play_apps_df['category'])
#new_col_names = mlb.classes_

# Create new DataFrame with transformed/one-hot encoded categories
#categories = pd.DataFrame(mlb.fit_transform(play_apps_df['category']), columns=new_col_names)

# Concat with original `category` column
#play_apps_df = pd.concat([play_apps_df, categories], axis=1 )
#play_apps_df.drop('category', axis=1,inplace=True)
#---------------------------------------------------


#Temporary removed columns
play_apps_df.drop(['bids'], axis=1, inplace=True)
play_apps_df.drop(['content_rating'], axis=1, inplace=True)
play_apps_df.drop(['interactive_elements'], axis=1, inplace=True)
play_apps_df.drop('category', axis=1,inplace=True)


play_apps_df

,video,score,reviews,editors_choice,price,iap,installs,app_id
0,0,-0.482481,-0.100461,False,-0.037445,False,-0.136503,a008.com.fc2.blog.androidkaihatu.datecamera2
1,0,0.987413,-0.099059,False,-0.037445,False,-0.139379,a201706011153.xsky.txvpn
2,0,0.987413,-0.099157,False,-0.037445,False,-0.139379,a201706021616.vpn.turbovpn
3,1,-0.727463,-0.101047,False,-0.037445,False,-0.142198,a201707.grmo.a8bit.jp.beautytimer
4,0,0.007483,-0.099509,False,-0.037445,False,-0.136503,a2x.studio.fast.charging.battery.supercharging
...,...,...,...,...,...,...,...,...
29458,1,0.742430,3.432597,True,-0.037445,True,2.734284,zombie.survival.craft.z
29459,0,-0.727463,-0.100831,False,-0.037445,False,-0.141681,zombi.maps.forminecraftpe
29460,0,0.497448,-0.098119,False,-0.037445,False,-0.113491,zone.super.jungle.worldrun2018
29461,1,0.987413,-0.063179,False,-0.037445,True,-0.084725,zoo.rescue


# Merge two datasets

In [17]:
print('---------------------------------------------')
#merge two data sets into merged dataframe
merged_df = pd.merge(android_df, play_apps_df, on='app_id')
merged_df = merged_df.drop(['app_id'], axis=1)

print('Rows before merge: ',android_df.shape[0])
print('Rows after merge: ', merged_df.shape[0])

#Concolusion after merge
# from android = 2936921 rows × 10 columns turn into 2656577 rows × 24 columns
# we missed ~300k rows. need to check y.....
print('---------------------------------------------')


merged_df




---------------------------------------------
Rows before merge:  2936921
Rows after merge:  2656577
---------------------------------------------


,utc_time,user_state,click,screen_resolution,video,score,reviews,editors_choice,price,iap,installs
0,1536444331894,43,0,-0.530933,1,0.252466,0.117100,False,-0.037445,True,0.433052
1,1536458660352,14,0,-0.530933,1,0.252466,0.117100,False,-0.037445,True,0.433052
2,1536458657259,9,0,3.294503,1,0.252466,0.117100,False,-0.037445,True,0.433052
3,1535716938954,9,0,-0.530933,1,0.252466,0.117100,False,-0.037445,True,0.433052
4,1536458532674,9,0,3.294503,1,0.252466,0.117100,False,-0.037445,True,0.433052
...,...,...,...,...,...,...,...,...,...,...,...
2656572,1534378690972,13,0,-1.147553,1,0.742430,-0.086263,False,-0.037445,True,-0.084725
2656573,1541459326798,24,0,-0.530933,0,-0.482481,-0.101058,False,-0.037445,True,-0.142198
2656574,1534305772980,12,0,0.857330,0,0.497448,-0.100489,False,-0.037445,True,-0.136503
2656575,1537471041561,9,0,-0.530933,0,1.477377,0.443755,False,-0.037445,False,0.433052


# Split data and save to files

In [18]:
print('---------------------------------------------')
print('Amount of NaN under utc_time: ', merged_df['utc_time'].isnull().sum())
print('Amount of NaN in all data : ', merged_df.isnull().sum().sum())
print('---------------------------------------------')


print('---------------------------------------------')
#Print start time after merge
merged_start_time = merged_df['utc_time'].min()
merged_end_time = merged_df['utc_time'].max()
print('Start Time: ',  datetime.fromtimestamp(merged_start_time/1000))
print('End Time: ',  datetime.fromtimestamp(merged_end_time/1000))
print('Split time: ',  datetime.fromtimestamp(1541023200000/1000))
print('---------------------------------------------')


print('---------------------------------------------')
tarin_df, test_df = merged_df[(mask:=merged_df['utc_time'] < 1541023200000)], merged_df[~mask]

tarin_df = tarin_df.drop(['utc_time'], axis=1)
test_df = test_df.drop(['utc_time'], axis=1)

tarin_size = tarin_df.shape[0]
test_size = test_df.shape[0]

print('Amount of tarin_df data : ', tarin_size)
print('Amount of test_df data : ', test_size)

print(f'Test data is: {(test_size / (tarin_size + test_size))*100} percent')

      
print('---------------------------------------------')
      
#Save to CSV files
#Save to splited csv files
filepath1 = Path('data/train.csv')  
filepath2 = Path('data/test.csv')  
tarin_df.to_csv(filepath1, index=False)
test_df.to_csv(filepath2, index=False)



---------------------------------------------
Amount of NaN under utc_time:  0
Amount of NaN in all data :  0
---------------------------------------------
---------------------------------------------
Start Time:  2018-08-06 21:00:20.974000
End Time:  2018-11-19 14:24:45.833000
Split time:  2018-11-01 00:00:00
---------------------------------------------
---------------------------------------------
Amount of tarin_df data :  2291793
Amount of test_df data :  364784
Test data is: 13.731354295395917 percent
---------------------------------------------


# SMOTE balancing

In [ ]:
# Only if not installed
#!pip install imbalanced-learn
#!pip install delayed

In [ ]:
# Trying to eliminate not balanced clicks
from imblearn.over_sampling import SMOTE

oversample_smote = SMOTE()
X_smote, y_smote = oversample_smote.fit_resample(merged_df.drop('click', axis=1), merged_df['click'])

y_smote

In [ ]:
X_smote

In [ ]:
# Creating a new DataFrame from SMOTE

merged_smote_df = X_smote.copy()
merged_smote_df['click'] =  y_smote
train_smote_df, test_smote_df = merged_df[(mask:=merged_smote_df['utc_time'] < 1541023200000)], merged_smote_df[~mask]

train_smote_df = train_smote_df.drop(['utc_time'], axis=1)
test_smote_df = test_smote_df.drop(['utc_time'], axis=1)

print('Amount of train SMOTE_df data : ', train_smote_df.shape[0])
print('Amount of test SMOTE data : ', test_smote_df.shape[0])

print('---------------------------------------------')
      
#Save to CSV files
#Save to splited csv files
filepath1 = Path('data/train_SMOTE.csv')  
filepath2 = Path('data/test_SMOTE.csv')  
train_smote_df.to_csv(filepath1, index=False)
test_smote_df.to_csv(filepath2, index=False)
